In [95]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl

import pandas as pd
import numpy as np

from kaggle.api.kaggle_api_extended import KaggleApi
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy import Column, Integer, String, Date, Float, create_engine, inspect, text

In [96]:
root_dir = os.getcwd()

kaggle_data_set = 'henrysue/online-shoppers-intention'

dataset_dir = 'dataset'
dataset_name = 'online_shoppers_intention.csv'
dataset_path = os.path.join(root_dir, dataset_dir, dataset_name)

separator = f"\n{'-'*100}\n"

In [97]:
def load_data_csv(file_path):
    """
    Загрузка данных из CSV файла.
    :param file_path: Путь к CSV файлу.
    :return: DataFrame с загруженными данными.
    """
    try:
        return pd.read_csv(file_path)
    except Exception as e:
        raise Exception(e)

In [98]:
def load_kaggle_data_set(dataset):
    """
    Загрузка датасета из kaggle.

    Важно: Необходимо предварительно получить API Token.
    Переместите файл kaggle.json в папку ~/.kaggle/ (для Linux и macOS)
    или в C:/Users/<Ваше_имя_пользователя>/.kaggle/ (для Windows).
    Если папка .kaggle не существует, создайте ее.
    :param dataset:
    :return:
    """

    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files(dataset=dataset,
                               path=f'./{dataset_dir}',
                               force=True,
                               unzip=True)              # Загрузка и распаковка

In [99]:
try:
    if not os.path.exists(dataset_path):
        # Загрузка файла датасета из kaggle если он не загружен
        load_kaggle_data_set(kaggle_data_set)
        # Загрузка файла в датафрейм
    df = load_data_csv(dataset_path)
except Exception as e:
    raise Exception (f'При выполнении произошла ошибка: {e}')

In [102]:
# Экспорт датасета в Exel если не был создан ранее
dataset_exel = os.path.join(root_dir, 'dataset.xlsx')
print(dataset_exel)
if not os.path.isfile(dataset_exel):
    df.to_excel(dataset_exel, index=False)

D:\Projects\My-education\BELHARD\DataScience\BaseLevel\HomeWork\DZ-4\dataset.xlsx


In [90]:
# Информация о датафрейме (датасете)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

In [ ]:
df.

In [91]:
# Вывод информации о дубликатах
print(separator, 'Количество дублирующихся значений: ', df.duplicated().sum())


----------------------------------------------------------------------------------------------------
 Количество дублирующихся значений:  125


In [92]:
# Убрать дубликаты записей
df.drop_duplicates(inplace=True)

In [93]:
# Информация о датафрейме (датасете)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12205 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12205 non-null  int64  
 1   Administrative_Duration  12205 non-null  float64
 2   Informational            12205 non-null  int64  
 3   Informational_Duration   12205 non-null  float64
 4   ProductRelated           12205 non-null  int64  
 5   ProductRelated_Duration  12205 non-null  float64
 6   BounceRates              12205 non-null  float64
 7   ExitRates                12205 non-null  float64
 8   PageValues               12205 non-null  float64
 9   SpecialDay               12205 non-null  float64
 10  Month                    12205 non-null  object 
 11  OperatingSystems         12205 non-null  int64  
 12  Browser                  12205 non-null  int64  
 13  Region                   12205 non-null  int64  
 14  TrafficType              12

In [94]:
# Отсутствующие значения в столбцах датафрейма. Можно посмотреть и в информации, но так более наглядно.
print(separator, 'Отсутствующие значения:\n', df.isna().sum())


----------------------------------------------------------------------------------------------------
 Отсутствующие значения:
 Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64
